# The revised simplex method

## Table of contents

## Introduction

The revised simplex method is commonly use method

## Example Problems With Worked Solutions

<details>
    <summary> Problem 1 </summary>

### Problem Statement

Maximize:

$\quad z = 4x_1 + 3x_2 + x_3 + 7x_4 + 6x_5$

Subject to:

$
\begin{array}{rl}
x_1 + 2x_2 + 3x_3 + x_4 - 3x_5 + s_1 & = 9, \\
2x_1 - x_2 + 2x_3 + 2x_4 + x_5 + s_2 & = 10, \\
-3x_1 + 2x_2 + x_3 - x_4 + 2x_5 + s_3 & = 11,
\end{array}
$

Where $x \geq 0, \, s \geq 0$.

<details>
    <summary>Click to view the complete solution</summary>

<details>
    <summary>Step 0 (Initialization)</summary>

\[
\mathbf{x_B} = \begin{pmatrix} s_1, s_2, s_3 \end{pmatrix}^T, \quad 
\mathbf{x_N} = \begin{pmatrix} x_1, x_2, x_3, x_4, x_5 \end{pmatrix}^T, \quad 
\mathbf{c_B^T} = (0, 0, 0), \quad 
\mathbf{c_N^T} = (4, 3, 1, 7, 6),
\]

\[
\mathbf{b} = \begin{pmatrix} 9 \\ 10 \\ 11 \end{pmatrix}, \quad 
\mathbf{B^{-1}} = \mathbf{I}, \quad 
\mathbf{u^T} = \mathbf{c_B^T B^{-1}} = (0, 0, 0),
\]

\[
\mathbf{\bar{b}} = \mathbf{B^{-1}b} = \begin{pmatrix} 9 \\ 10 \\ 11 \end{pmatrix}, \quad 
z = 0.
\]
</details>

<details>
    <summary>Iteration 1</summary>

Compute:
\[
\mathbf{c_N^T} - \mathbf{c_B^T B^{-1} N} = (-4, -3, -1, -7, -6).
\]
Select $x_4$ as the entering variable.

Then, the pivoting column 4 is calculated by:
\[
\mathbf{B^{-1}a_4} = 
\begin{pmatrix} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 \end{pmatrix}
\begin{pmatrix} 1 \\ 2 \\ -1 \end{pmatrix} 
= \begin{pmatrix} 1 \\ 2 \\ -1 \end{pmatrix},
\]
\[
\mathbf{c_4^T a_4} = -7.
\]

Add the pivot column to the right of the revised simplex tableau, and $s_2$ becomes the leaving variable.

**Tableau after pivoting:**
\[
\begin{array}{c|cccc|c}
& x_1 & x_2 & x_3 & x_4 & \text{RHS} \\
\hline
z & 0 & 3.5 & 0 & 35 & 0 \\
s_1 & 1 & -0.5 & 0 & 4 & 0 \\
x_4 & 0 & 0.5 & 0 & 5 & 1 \\
s_3 & 0 & 0.5 & 1 & 16 & 0 \\
\end{array}
\]
</details>

<details>
    <summary>Iteration 2</summary>

$u^T = (0, 3.5, 0)$.

\[
\mathbf{c_N^T} - \mathbf{c_B^T B^{-1} N} = (3, -6.5, 6, 0, -2.5, 0, 3.5, 0).
\]
Select $x_2$ as the entering variable.

Then, the pivoting column is calculated:
\[
\mathbf{B^{-1}a_2} = 
\begin{pmatrix} 1 & -0.5 & 0 \\ 0 & 0.5 & 0 \\ 0 & 0.5 & 1 \end{pmatrix}
\begin{pmatrix} 2 \\ -1 \\ 2 \end{pmatrix} 
= \begin{pmatrix} 2.5 \\ -0.5 \\ 1.5 \end{pmatrix},
\]
\[
\mathbf{c_2^T a_2} = -6.5.
\]

$s_1$ becomes the leaving variable.

**Tableau after pivoting:**
\[
\begin{array}{c|cccc|c}
& x_1 & x_2 & x_3 & x_4 & \text{RHS} \\
\hline
z & 2.6 & 2.2 & 0 & 45.4 & 0 \\
x_2 & 0.4 & -0.2 & 0 & 1.6 & 1 \\
x_4 & 0.2 & 0.4 & 0 & 5.8 & 0 \\
s_3 & -0.6 & 0.8 & 1 & 13.6 & 0 \\
\end{array}
\]
</details>

<details>
    <summary>Iteration 3</summary>

$u^T = (2.6, 2.2, 0)$.

\[
\mathbf{c_N^T} - \mathbf{c_B^T B^{-1} N} = (3, 0, 11.2, 0, -11.6, 2.6, 2.2, 0).
\]
Select $x_5$ as the entering variable.

The pivot column is calculated:
\[
\mathbf{B^{-1}a_5} = 
\begin{pmatrix} 0.4 & -0.2 & 0 \\ 0.2 & 0.4 & 0 \\ -0.6 & 0.8 & 1 \end{pmatrix}
\begin{pmatrix} -3 \\ 1 \\ 2 \end{pmatrix} 
= \begin{pmatrix} -1.4 \\ -0.2 \\ 4.6 \end{pmatrix},
\]
\[
\mathbf{c_5^T a_5} = -11.6.
\]

$s_3$ becomes the leaving variable.

**Tableau after pivoting:**
\[
\begin{array}{c|cccc|c}
& x_1 & x_2 & x_3 & x_4 & \text{RHS} \\
\hline
z & 1.09 & 4.22 & 2.52 & 79.7 & 0 \\
x_2 & 0.22 & -0.04 & 0.30 & 5.74 & 1 \\
x_4 & 0.17 & 0.43 & 0.04 & 6.39 & 0 \\
x_5 & -0.13 & 0.17 & 0.22 & 2.96 & 0 \\
\end{array}
\]
</details>

<details>
    <summary>Iteration 4</summary>

$u^T = (1.09, 4.22, 2.52)$.

\[
\mathbf{c_N^T} - \mathbf{c_B^{-1} N} = (-2.04, 0, 13.22, 0, 1.09, 4.22, 2.52).
\]
Select $x_1$ as the entering variable.

The pivot column is calculated:
\[
\mathbf{B^{-1}a_1} = 
\begin{pmatrix} 0.22 & -0.04 & 0.30 \\ 0.17 & 0.43 & 0.04 \\ -0.13 & 0.17 & 0.22 \end{pmatrix}
\begin{pmatrix} 1 \\ 2 \\ -3 \end{pmatrix} 
= \begin{pmatrix} -0.61 \\ 0.913 \\ -0.43 \end{pmatrix},
\]
\[
\mathbf{c_1^T a_1} = -2.04.
\]

$x_4$ becomes the leaving variable.

**Tableau after pivoting:**
\[
\begin{array}{c|cccc|c}
& x_1 & x_2 & x_3 & x_5 & \text{RHS} \\
\hline
z & 1.48 & 5.19 & 2.62 & 94 & 0 \\
x_2 & 0.33 & 0.33 & 0.33 & 10 & 0 \\
x_1 & 0.19 & 0.48 & 0.05 & 7 & 1 \\
x_5 & -0.05 & 0.38 & 0.24 & 6 & 0 \\
\end{array}
\]
</details>

<details>
    <summary>Iteration 5</summary>

$u^T = (1.48, 5.19, 2.62)$.

\[
\mathbf{c_N^T} - \mathbf{c_B^{-1} N} = (0, 0, 16.42, 2.23, 0, 1.48, 5.19, 2.62).
\]
Since all reduced costs are non-negative, the optimal solution is reached.

**Final Tableau:**
\[
\begin{array}{c|cccc|c}
& x_1 & x_2 & x_3 & x_5 & \text{RHS} \\
\hline
z & 1.48 & 5.19 & 2.62 & 94 & 0 \\
x_2 & 0.33 & 0.33 & 0.33 & 10 & 0 \\
x_1 & 0.19 & 0.48 & 0.05 & 7 & 1 \\
x_5 & -0.05 & 0.38 & 0.24 & 6 & 0 \\
\end{array}
\]

**Optimal Solution:**
- $x_1 = 7$
- $x_2 = 10$
- $x_5 = 6$
- Maximum value of the objective function $z = 94$.

</details>
</details>
</details>


In [1]:
using LinearAlgebra
using SparseArrays
using SuiteSparse
using Random
using ArgParse

# local modules
push!(LOAD_PATH, realpath("../code"))
using lp_constants
using lp_utils
using lp_problem
using lp_read_mps

In [2]:
mps_filename = "../../benchmarks/mps_files/test.mps"
lp = read_mps_from_file(mps_filename)
print(lp)

LPProblem(false, [-110.0, -120.0, -130.0, -110.0, -115.0, 150.0], sparse([3, 5, 3, 5, 3, 5, 3, 5, 3, 5, 4], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6], [8.8, 1.0, 6.1, 1.0, 2.0, 1.0, 4.2, 1.0, 5.0, 1.0, 3.0], 5, 6), [200.0, 250.0, 0.0, -0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [Inf, Inf, Inf, Inf, Inf, Inf], ["VEG01", "VEG02", "OIL01", "OIL02", "OIL03", "PROD"], ['L', 'L', 'L', 'G', 'E'])

In [3]:
function convert_to_standard_form(lp::LPProblem)
    c, A, b, l, u = lp.c, lp.A, lp.b, lp.l, lp.u
    m, n = size(A)
    
    # Handle lower and upper bounds
    for i in 1:n
        if l[i] > -Inf
            A = [A; zeros(1, n)]
            A[end, i] = -1
            push!(b, -l[i])
            push!(lp.constraint_types, 'L')
        end
        if u[i] < Inf
            A = [A; zeros(1, n)]
            A[end, i] = 1
            push!(b, u[i])
            push!(lp.constraint_types, 'L')
        end
    end
    
    m, n = size(A)
    new_A = spzeros(eltype(A), m, n + m)
    new_b = copy(b)
    new_c = [c; zeros(m)]
    
    for i in 1:m
        if lp.constraint_types[i] == 'L'
            new_A[i, :] = [A[i, :]; zeros(i-1); 1; zeros(m-i)]
        elseif lp.constraint_types[i] == 'G'
            new_A[i, :] = [-A[i, :]; zeros(i-1); 1; zeros(m-i)]
            new_b[i] = -new_b[i]
        elseif lp.constraint_types[i] == 'E'
            new_A[i, :] = [A[i, :]; zeros(m)]
        end
    end
    
    if !lp.is_minimize
        new_c = -new_c
    end

    # Return the new LPProblem in standard form
    return LPProblem(lp.is_minimize, new_c, new_A, new_b, lp.l, lp.u, lp.vars, lp.constraint_types)
end


convert_to_standard_form (generic function with 1 method)

In [4]:
function revised_simplex(lp::LPProblem)
    println("Converting problem to standard form...")
    A, b, c = convert_to_standard_form(lp)
    m, n = size(A)
    
    println("\nStandard form problem:")
    println("  Objective function coefficients c: ", c)
    println("  Constraint matrix A: ", A)
    println("  Right-hand side b: ", b)
    println("  Variables: ", [lp.vars; ["s$i" for i in 1:(n-length(lp.vars))]])
    println("  Optimization type: ", lp.is_minimize ? "Minimize" : "Maximize")
    println("  constraint_types = $(lp.constraint_types)")
    
    # Initialize basis with slack variables
    B = collect((length(lp.vars)+1):n)
    N = collect(1:length(lp.vars))
    
    println("\nInitial basis: ", B)
    println("Initial non-basic variables: ", N)

    # Initialize B_factor outside the loop
    B_matrix = A[:, B]
    B_factor = lu(B_matrix)    
    
    iteration = 0
    while true
        iteration += 1
        println("\nIteration ", iteration)
        
        # Step 1: Compute basic solution
        B_matrix = A[:, B]
        x_B = B_matrix \ b
        println("  Basic solution x_B: ", x_B)
        
        # Step 2: Compute reduced costs
        y = (c[B]' / B_matrix)'
        c_N = c[N] - A[:, N]' * y
        println("  Dual variables y: ", y)
        println("  Reduced costs c_N: ", c_N)
        
        # Step 3: Check optimality
        if all(c_N .>= -1e-10)
            x = zeros(n)
            x[B] = x_B
            println("\nOptimal solution found:")
            obj_value = dot(lp.is_minimize ? c : -c, x)
            return x[1:length(lp.vars)], obj_value
        end
        
        # Step 4: Choose entering variable
        e = argmin(c_N)
        q = N[e]
        println("  Entering variable: ", q)
        
        # Step 5: Compute direction
        aq = A[:, q]  # Extract q-th column of A
        d = Vector(B_factor \ Vector(aq))  # Convert to dense, solve, and convert back
        println("  Direction d: ", d)
        
        # Step 6 & 7: Check unboundedness and choose leaving variable
        if all(d .<= 1e-10)
            error("Problem is unbounded")
        end
        
        ratios = x_B ./ d
        ratios[d .<= 1e-10] .= Inf
        valid_ratios = filter(x -> x > 0, ratios)
        if isempty(valid_ratios)
            error("Problem is unbounded")
        end
        l = argmin(valid_ratios)
        p = B[l]
        println("  Leaving variable: ", p)

        # Step 8: Update basis
        B[l] = q
        N[e] = p
        println("  New basis: ", B)
        println("  New non-basic variables: ", N)
        
        # Update B_factor when the basis changes
        B_matrix = A[:, B]
        B_factor = lu(B_matrix)
        
        if iteration > 10  # FIXME: Add a better termination criterion
            error("Maximum iterations reached")
        end
    end
end

revised_simplex (generic function with 1 method)

In [5]:
# Call the revised_simplex function
optimal_solution, optimal_value = revised_simplex(lp)
    
# Print the results
println("Optimal solution: ", optimal_solution)
println("Optimal objective value: ", optimal_value)

Converting problem to standard form...


LoadError: MethodError: no method matching iterate(::LPProblem)
[0mClosest candidates are:
[0m  iterate([91m::Union{LinRange, StepRangeLen}[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/range.jl:826
[0m  iterate([91m::Union{LinRange, StepRangeLen}[39m, [91m::Integer[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/range.jl:826
[0m  iterate([91m::T[39m) where T<:Union{Base.KeySet{<:Any, <:Dict}, Base.ValueIterator{<:Dict}} at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/dict.jl:695
[0m  ...

In [ ]:
for element in preprocessed_lp
    println(element)
end
